In [ ]:
import numpy as np
import torch
import argparse
import torch.nn.functional as F
from ogb.nodeproppred import Evaluator
from utils import set_seed, load_data, get_model

In [2]:
def parse_args():
    parser = argparse.ArgumentParser(description='GCN')
    parser.add_argument('--repetitions', type=int, default=10)
    parser.add_argument('--random_seed', type=int, default=10)
    parser.add_argument('--dataset', type=str, default='CoauthorPhysics')
    parser.add_argument('--device', type=int, default=0)
    parser.add_argument('--type_model', type=str, default='UGDGNN')   
    parser.add_argument('--transductive', type=bool, default=True)
    args = parser.parse_args(args=[])
    return args

In [3]:
def main(args):
    print(args)    
    best_acc_mean = 0
    best_acc_std = 0
    device = torch.device(f'cuda:{args.device}' if torch.cuda.is_available() else 'cpu')     
    list_test_acc = []
    list_valid_acc = []
    list_train_loss = []    
    for repetition in range(args.repetitions):
        print(f'Repetition <{repetition}>')
        set_seed(repetition)
        args, data = load_data(args)                
        data = data.to(device)
        args.num_features = data.num_node_features 
        torch.cuda.empty_cache()                                       
        model = get_model(args)               
        model.cuda(device) 
        best_train_loss = 100.
        best_val_loss = 100.
        best_train_acc = 0.
        best_val_acc = 0.        
        best_test_acc = 0.
        bad_counter = 0.
        for epoch in range(args.epochs):
            model.train()       
            output = model(data.x, data.edge_index)           
            loss = 0.
            
            if args.dataset == 'ogbn-arxiv':
                loss_train = F.nll_loss(output[data.train_mask], data.y.squeeze(1)[data.train_mask])
            else:
                loss_train = F.nll_loss(output[data.train_mask], data.y[data.train_mask])   

            model.optimizer.zero_grad()
            loss_train.backward()
            model.optimizer.step()

            model.eval()
            output = model(data.x, data.edge_index)
            if args.dataset == 'ogbn-arxiv':
                evaluator = Evaluator(name='ogbn-arxiv')
                acc_train = evaluator.eval({'y_true': data.y[data.train_mask],
                        'y_pred': torch.argmax(output, dim=1)[data.train_mask].reshape(len(data.train_mask),1),})['acc']
                acc_val = evaluator.eval({'y_true': data.y[data.val_mask],
                        'y_pred': torch.argmax(output, dim=1)[data.val_mask].reshape(len(data.val_mask),1),})['acc']
                acc_test = evaluator.eval({'y_true': data.y[data.test_mask],
                        'y_pred': torch.argmax(output, dim=1)[data.test_mask].reshape(len(data.test_mask),1),})['acc']
            else:
                acc_train = torch.sum(torch.argmax(output, dim=1)[data.train_mask] == 
                                      data.y[data.train_mask]).item() * 1.0 / data.train_mask.sum().item()
                acc_val = torch.sum(torch.argmax(output, dim=1)[data.val_mask] == 
                                      data.y[data.val_mask]).item() * 1.0 / data.val_mask.sum().item()
                acc_test = torch.sum(torch.argmax(output, dim=1)[data.test_mask] == 
                                      data.y[data.test_mask]).item() * 1.0 / data.test_mask.sum().item()

            if args.dataset == 'ogbn-arxiv':
                loss_val = F.nll_loss(output[data.val_mask], data.y.squeeze(1)[data.val_mask]) 
            else:
                loss_val = F.nll_loss(output[data.val_mask], data.y[data.val_mask])     

            if loss_val < best_val_loss:
                best_train_loss = loss_train
                best_val_loss = loss_val
                best_train_acc = acc_train
                best_val_acc = acc_val
                best_test_acc = acc_test               
                bad_counter = 0
            else:
                bad_counter += 1
            if bad_counter == args.patience:
                break
        print('test_acc:{:.4f}'.format(best_test_acc))
        list_train_loss.append(best_train_loss)
        list_valid_acc.append(best_val_acc)
        list_test_acc.append(best_test_acc)
    print('final mean and std of test acc with <{}> runs: {:.4f}±{:.4f}'.format(
        args.repetitions, np.mean(list_test_acc), np.std(list_test_acc)))

In [4]:
args = parse_args()

In [5]:
main(args)

Namespace(dataset='CoauthorPhysics', device=0, random_seed=10, repetitions=10, transductive=True, type_model='UGDGNN')
Repetition <0>


Processing...
Done!


test_acc:0.9315
Repetition <1>
test_acc:0.9386
Repetition <2>
test_acc:0.9368
Repetition <3>
test_acc:0.9436
Repetition <4>
test_acc:0.9423
Repetition <5>
test_acc:0.9451
Repetition <6>
test_acc:0.9460
Repetition <7>
test_acc:0.9480
Repetition <8>
test_acc:0.9366
Repetition <9>
test_acc:0.9454
final mean and std of test acc with <10> runs: 0.9414±0.0050
